In [1]:
#importing all the required libraries
import selenium
from selenium import webdriver
import time
import requests
import json
from bs4 import BeautifulSoup
import os
from PIL import Image
import io
import hashlib
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,ElementNotInteractableException,InvalidArgumentException,InvalidCookieDomainException

### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#connecting to the webdriver
driver = webdriver.Chrome(r"C:\Users\saurabh\Downloads\chromedriver_win32\chromedriver.exe")

In [3]:
#specifying the url of the webpage to be scraped
url="https://www.amazon.in/"

In [4]:
#opening the webpage through our driver
driver.get(url)

In [46]:
# Taking input from user 
search_string = input("Input the string you want to search for:")

#generate a url from search string
url= driver.get("https://www.amazon.in/s?k=" + search_string + "&start=")

Input the string you want to search for:Television


### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [50]:
#getting the list of url of televisions
TV_URL=[]

for r in range(0,3):
    television=driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']")
    for i in television:
        television2=i.find_element_by_tag_name("a")
        TV_URL.append(television2.get_attribute('href'))
try:        
    driver.find_element_by_xpath('//*[@id="search"]/div[1]/div[2]/div/span[3]/div[2]/div[27]/span/div/div/ul/li[3]/a').click()
except NoSuchElementException:
    driver.find_element_by_xpath('//*[@id="search"]/div[1]/div[2]/div/span[3]/div[2]/div[25]/span/div/div/ul/li[4]/a').click()

In [51]:
TV_URL

['https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg2_1?ie=UTF8&adId=A0720853R9IIRAIZ5A7S&url=%2FTCL-inches-Certified-Android-43P615%2Fdp%2FB08FD2VSD9%2Fref%3Dsr_1_17_sspa%3Fdchild%3D1%26keywords%3DTelevision%26qid%3D1612271859%26sr%3D8-17-spons%26psc%3D1&qualifier=1612271873&id=1126860968512705&widgetName=sp_atf_next',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg2_1?ie=UTF8&adId=A1011788PE2O1M5RBTM2&url=%2FSamsung-inches-Wondertainment-Ready-UA32TE40AAKBXL%2Fdp%2FB08PV1X771%2Fref%3Dsr_1_18_sspa%3Fdchild%3D1%26keywords%3DTelevision%26qid%3D1612271859%26sr%3D8-18-spons%26psc%3D1&qualifier=1612271873&id=1126860968512705&widgetName=sp_atf_next',
 'https://www.amazon.in/TCL-100-3cm-inches-Android-Smart/dp/B07KCCWJTP/ref=sr_1_19?dchild=1&keywords=Television&qid=1612271859&sr=8-19',
 'https://www.amazon.in/Shinco-Inches-Ready-SO32SF-Built/dp/B08QRDY1FD/ref=sr_1_20?dchild=1&keywords=Television&qid=1612271859&sr=8-20

In [52]:
#checking the length of tv url
len(TV_URL)

66

In [53]:
#going to the url of first television through our driver
driver.get(TV_URL[0])

In [54]:
#creating the empty lists
Brand_name=[]
Name_of_product=[]
Rating=[]
No_of_ratings=[]
Price=[]
Return_exchange=[]
Expected_delivery=[]
Availibility=[]
Other_details=[]

In [55]:
#running a loop to extract all the required information from TV_URL

from tqdm import tqdm
for i in tqdm(TV_URL):
    driver.get(i)
    try:
        brand=driver.find_element_by_xpath('//*[@id="bylineInfo"]').text
        Brand_name.append(brand)
    except NoSuchElementException:
        Brand_name.append('-')
        
    try:
        product=driver.find_element_by_xpath('//*[@id="productTitle"]').text
        Name_of_product.append(product)
    except NoSuchElementException:
        Name_of_product.append('-')
    
    try:
        rating_=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").click()
        rating=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']").text
        Rating.append(rating)
    except NoSuchElementException as e:
        Rating.append('-')
        
    try:
        ratings=driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
        No_of_ratings.append(ratings)
    except NoSuchElementException:
        No_of_ratings.append('-')
        
    try:
        price=driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text
        Price.append(price)
    except NoSuchElementException:
        Price.append('-')
        
    try:
        exchange=driver.find_element_by_xpath('//*[@id="icon-farm-container"]/div/div[1]/span/div[2]/a').text
        Return_exchange.append(exchange)
    except NoSuchElementException:
        Return_exchange.append('-')
        
    try:
        delivery=driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text
        Expected_delivery.append(delivery)
    except NoSuchElementException:
        Expected_delivery.append('-')
        
    try:
        availibility=driver.find_element_by_xpath('//*[@id="availability"]/span').text
        Availibility.append(availibility)
    except NoSuchElementException:
        Availibility.append('-')
        
    try:
        details=driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul').text.replace("\n"," ")
        Other_details.append(details)
    except NoSuchElementException:
        Other_details.append('-')
        
    time.sleep(3)

100%|██████████| 66/66 [08:24<00:00,  7.65s/it]


In [56]:
Brand_name

['Visit the TCL Store',
 'Visit the Samsung Store',
 'Visit the TCL Store',
 'Visit the Shinco Store',
 'Visit the TCL Store',
 'Visit the Kodak Store',
 'Brand: Visio World',
 'Visit the eAirtec Store',
 'Brand: MI',
 'Visit the Kevin Store',
 'Visit the Panasonic Store',
 'Visit the TCL Store',
 'Visit the Kevin Store',
 'Visit the TCL Store',
 'Brand: VU',
 'Visit the Samsung Store',
 'Brand: Sony',
 'Visit the Onida Store',
 'Brand: AmazonBasics',
 'Visit the Sanyo Store',
 'Brand: VU',
 'Visit the Sanyo Store',
 'Visit the TCL Store',
 'Visit the Samsung Store',
 'Visit the TCL Store',
 'Visit the Shinco Store',
 'Visit the TCL Store',
 'Visit the Kodak Store',
 'Brand: Visio World',
 'Visit the eAirtec Store',
 'Brand: MI',
 'Visit the Kevin Store',
 'Visit the Panasonic Store',
 'Visit the TCL Store',
 'Visit the Kevin Store',
 'Visit the TCL Store',
 'Brand: VU',
 'Visit the Samsung Store',
 'Brand: Sony',
 'Visit the Onida Store',
 'Brand: AmazonBasics',
 'Visit the Sanyo Stor

In [57]:
Name_of_product

['TCL 108 cm (43 inches) 4K Ultra HD Certified Android Smart LED TV 43P615 (Black) (2020 Model)',
 'Samsung 80 cm (32 inches) Wondertainment Series HD Ready LED Smart TV UA32TE40AAKBXL (Titan Gray) (2021 model)',
 'TCL 100 cm (40 inches) Full HD Certified Android Smart LED TV 40S6500FS (Black) (2020 Model)',
 'Shinco 80 cm (32 Inches) HD Ready Smart LED TV SO32SF (Black) (2021 Model) | with Alexa Built-in',
 'TCL 108 cm (43 inches) 4K Ultra HD Certified Android Smart LED TV 43P615 (Black) (2020 Model)',
 'Kodak 80 cm (32 Inches) HD Certified Android LED TV 32HDX7XPRO (Black) (2020 Model)',
 'VW 80 cm (32 inches) HD Ready LED Smart TV VW32PRO (Black) (2021 Model)',
 'eAirtec 81 cm (32 inches) HD Ready LED TV 32DJ (Black) (2020 Model)',
 'MI TV 4A 100 cm (40 Inches) Full HD Android LED TV (Black) | with Data Saver',
 'Kevin 80 cm (32 Inches) HD Ready LED TV K56U912 (Black) (2021 Model)',
 'Panasonic 147 cm (58 inches) 4K Ultra HD Certified Android Smart LED TV TH-58HX450DX (Black) (2020 

In [58]:
Rating

['4.2 out of 5',
 '4.3 out of 5',
 '4.1 out of 5',
 '3.8 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.5 out of 5',
 '3.9 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.1 out of 5',
 '4.1 out of 5',
 '5 out of 5',
 '4.1 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.7 out of 5',
 '4.3 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.1 out of 5',
 '3.8 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.5 out of 5',
 '3.9 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.1 out of 5',
 '4.1 out of 5',
 '5 out of 5',
 '4.1 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.7 out of 5',
 '4.3 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.1 out of 5',
 '3.8 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '4.5 out of 5',
 '3.9 out of 5',
 '4.2 out of 5',
 '4.3 out of 5',
 '4.1 out of 5',
 '4.1 out of 5',
 '5 out of 5',
 '4.1 out of 5',
 '4.2 out of 5',
 '4

In [59]:
No_of_ratings

['289 ratings',
 '1,678 ratings',
 '823 ratings',
 '11 ratings',
 '289 ratings',
 '4,545 ratings',
 '33 ratings',
 '1,919 ratings',
 '29,917 ratings',
 '6,996 ratings',
 '151 ratings',
 '823 ratings',
 '9 ratings',
 '160 ratings',
 '8,181 ratings',
 '1,678 ratings',
 '2,819 ratings',
 '3,475 ratings',
 '45 ratings',
 '1,000 ratings',
 '8,181 ratings',
 '1,000 ratings',
 '289 ratings',
 '1,678 ratings',
 '823 ratings',
 '11 ratings',
 '289 ratings',
 '4,545 ratings',
 '33 ratings',
 '1,919 ratings',
 '29,917 ratings',
 '6,996 ratings',
 '151 ratings',
 '823 ratings',
 '9 ratings',
 '160 ratings',
 '8,181 ratings',
 '1,678 ratings',
 '2,819 ratings',
 '3,475 ratings',
 '45 ratings',
 '1,000 ratings',
 '8,181 ratings',
 '1,000 ratings',
 '289 ratings',
 '1,678 ratings',
 '823 ratings',
 '11 ratings',
 '289 ratings',
 '4,545 ratings',
 '33 ratings',
 '1,919 ratings',
 '29,920 ratings',
 '6,996 ratings',
 '151 ratings',
 '823 ratings',
 '9 ratings',
 '160 ratings',
 '8,181 ratings',
 '1,678

In [60]:
Price

['₹ 26,990.00',
 '₹ 17,290.00',
 '₹ 19,999.00',
 '₹ 12,499.00',
 '₹ 26,990.00',
 '₹ 13,499.00',
 '₹ 11,290.00',
 '₹ 9,500.00',
 '₹ 19,999.00',
 '₹ 9,999.00',
 '₹ 47,990.00',
 '₹ 19,999.00',
 '₹ 12,999.00',
 '-',
 '₹ 25,999.00',
 '₹ 17,290.00',
 '₹ 34,990.00',
 '₹ 23,149.00',
 '₹ 23,499.00',
 '₹ 23,499.00',
 '₹ 25,999.00',
 '₹ 23,499.00',
 '₹ 26,990.00',
 '₹ 17,290.00',
 '₹ 19,999.00',
 '₹ 12,499.00',
 '₹ 26,990.00',
 '₹ 13,499.00',
 '₹ 11,290.00',
 '₹ 9,500.00',
 '₹ 19,999.00',
 '₹ 9,999.00',
 '₹ 47,990.00',
 '₹ 19,999.00',
 '₹ 12,999.00',
 '-',
 '₹ 25,999.00',
 '₹ 17,290.00',
 '₹ 34,990.00',
 '₹ 23,149.00',
 '₹ 23,499.00',
 '₹ 23,499.00',
 '₹ 25,999.00',
 '₹ 23,499.00',
 '₹ 26,990.00',
 '₹ 17,290.00',
 '₹ 19,999.00',
 '₹ 12,499.00',
 '₹ 26,990.00',
 '₹ 13,499.00',
 '₹ 11,290.00',
 '₹ 9,500.00',
 '₹ 19,999.00',
 '₹ 9,999.00',
 '₹ 47,990.00',
 '₹ 19,999.00',
 '₹ 12,999.00',
 '-',
 '₹ 25,999.00',
 '₹ 17,290.00',
 '₹ 34,990.00',
 '₹ 23,149.00',
 '₹ 23,499.00',
 '₹ 23,499.00',
 '₹ 25,999.0

In [61]:
Return_exchange

['10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replacement',
 '10 Days Replac

In [62]:
Expected_delivery

['Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Saturday, Feb 6, 8AM-5PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Saturday, Feb 6, 8AM-5PM',
 'Thursday, Feb 4, 8AM-2PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Saturday, Feb 6, 8AM-5PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Saturday, Feb 6, 8AM-5PM',
 'Thursday, Feb 4, 8AM-2PM',
 '-',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2PM',
 'Thursday, Feb 4, 8AM-2

In [63]:
Availibility

['In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'Available from these sellers.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'Available from these sellers.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'Available from these sellers.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.']

In [64]:
Other_details

["Resolution: 4K Ultra HD (3860 x 2160) | Refresh Rate: 60 Hertz Connectivity : 3 HDMI ports to connect set top box, Blue Ray players | 1 USB ports to connect hard drives and other USB devices Sound: 24 Watts Output | Dolby Audio Power Speakers | Surround Virtualizer Smart TV Features : Built-In Wifi | Amazon Prime video | Hands free voice control |4K Android TV+HDR 10 |AI-Google Assistant | Certified Android P| Screen Cast+T-cast |2GB RAM - 16GB ROM |5000+ Apps YouTube, Netflix Display : A+ Grade Panel | UHD | HDR 10 | Full Screen | Slim Design Warranty: 18 Months comprehensive warranty provided by TCL from date of purchase Installation: Brand will contact for installation for this product once delivered. Contact TCL Support for assistance @ 1800-419-0622 and provide product's model name and seller's details mentioned on your invoice. The service center will allot you a convenient slot for the service Easy returns: This product is eligible for replacement within 10 days of delivery in

In [65]:
#printing length
print(len(Brand_name),len(Name_of_product),len(Rating),len(No_of_ratings),len(Price),len(Return_exchange),len(Expected_delivery),len(Availibility),len(Other_details),len(TV_URL))

66 66 66 66 66 66 66 66 66 66


In [66]:
#creating a dataframe

Televisions=pd.DataFrame({})
Televisions['Brand_name']=Brand_name
Televisions['Name_of_product']=Name_of_product
Televisions['Rating']=Rating
Televisions['No_of_ratings']=No_of_ratings
Televisions['Price']=Price
Televisions['Return_exchange']=Return_exchange
Televisions['Expected_delivery']=Expected_delivery
Televisions['Availibility']=Availibility
Televisions['Other_details']=Other_details
Televisions['Product_URL']=TV_URL

In [67]:
Televisions

,Brand_name,Name_of_product,Rating,No_of_ratings,Price,Return_exchange,Expected_delivery,Availibility,Other_details,Product_URL
0,Visit the TCL Store,TCL 108 cm (43 inches) 4K Ultra HD Certified A...,4.2 out of 5,289 ratings,"₹ 26,990.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Resolution: 4K Ultra HD (3860 x 2160) | Refres...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Samsung Store,Samsung 80 cm (32 inches) Wondertainment Serie...,4.3 out of 5,"1,678 ratings","₹ 17,290.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Resolution: HD Ready (1366x768) | Refresh Rate...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the TCL Store,TCL 100 cm (40 inches) Full HD Certified Andro...,4.1 out of 5,823 ratings,"₹ 19,999.00",10 Days Replacement,"Saturday, Feb 6, 8AM-5PM",In stock.,Resolution: Full HD (1920 x 1080) | Refresh Ra...,https://www.amazon.in/TCL-100-3cm-inches-Andro...
3,Visit the Shinco Store,Shinco 80 cm (32 Inches) HD Ready Smart LED TV...,3.8 out of 5,11 ratings,"₹ 12,499.00",10 Days Replacement,-,In stock.,Resolution: HD Ready (1366x768) | Refresh Rate...,https://www.amazon.in/Shinco-Inches-Ready-SO32...
4,Visit the TCL Store,TCL 108 cm (43 inches) 4K Ultra HD Certified A...,4.2 out of 5,289 ratings,"₹ 26,990.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Resolution: 4K Ultra HD (3860 x 2160) | Refres...,https://www.amazon.in/TCL-inches-Certified-And...
...,...,...,...,...,...,...,...,...,...,...
61,Visit the Onida Store,Onida 108 cm (43 Inches) Fire TV Edition Full ...,4.3 out of 5,"3,476 ratings","₹ 23,149.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Fire TV experience built-in brings together yo...,https://www.amazon.in/Onida-108-Inches-Full-Sm...
62,Brand: AmazonBasics,AmazonBasics 109cm (43 inches) Fire TV Edition...,4.2 out of 5,45 ratings,"₹ 23,499.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Resolution: Full HD (1920x1080) | Refresh Rate...,https://www.amazon.in/AmazonBasics-108cm-inch-...
63,Visit the Sanyo Store,Sanyo 108 cm (43 inches) Kaizen Series Full HD...,4.4 out of 5,"1,000 ratings","₹ 23,499.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Resolution: Full HD (1920x1080) Refresh Rate: ...,https://www.amazon.in/Sanyo-inches-Certified-A...
64,Brand: VU,Vu 108 cm (43 inches) Full HD UltraAndroid LED...,4.2 out of 5,"8,181 ratings","₹ 25,999.00",10 Days Replacement,"Thursday, Feb 4, 8AM-2PM",In stock.,Resolution: Full HD (1920x1080) | Refresh Rate...,https://www.amazon.in/gp/slredirect/picassoRed...


In [68]:
#converting into csv format
Televisions.to_csv("Televisions.csv")

In [69]:
#closing the driver
driver.close()

### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [70]:
#Scraping images of fruits and machine learning


#setting driver path
DRIVER_PATH = r"C:\Users\saurabh\Downloads\chromedriver_win32\chromedriver.exe"


def fetch_image_urls(query:str, max_links_to_fetch:100, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ["fruits","machine learning"]  #set of querries
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,100,wd)
        #give images path
        images_path = r'C:\Users\saurabh\OneDrive\Desktop\New folder (3)'
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://www.verywellhealth.com/thmb/0mTCeKqywiovRWMiugc4mN99CGQ=/2880x1918/filters:fill(87E3EF,1)/fruits-to-avoid-if-you-have-diabetes-1087587-primary-recirc-3a95a09a48cb46b49d5632326f9405d3.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\da85f7a89d.jpg
SUCCESS - saved https://www.lalpathlabs.com/blog/wp-content/uploads/2019/01/Fruits-and-Vegetables.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\b1c5795b43.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRhp8LkNwuxtHEr7Hdz2eEFp0pzUcpQZhkJDw&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\096904f730.jpg
SUCCESS - saved https://www.verywellfit.com/thmb/a4580FjTjbub9q4kI5m9X-Po-p0=/2002x1334/filters:no_upscale():max_bytes(150000):strip_icc()/Bananas-5c6a36a346e0fb0001f0e4a3.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\865b67b8b6.jpg
SUCCES

SUCCESS - saved https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/articles/health_tools/exotic_fruit_slideshow/thinkstock_photo_of_grenadilla_passion_fruit.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\f016a13f22.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQndGzs9OXvCv1BC4j550PhbxjG6jvEq0xFOQ&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\52ed4e0a1f.jpg
SUCCESS - saved https://i2.wp.com/www.vegrecipesofindia.com/wp-content/uploads/2014/12/Indian-Fruits-Names.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\66f9b14e15.jpg
SUCCESS - saved https://hips.hearstapps.com/hmg-prod/images/fruit-salad-horizontal-jpg-1522181219.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\5dd8bf96d9.jpg
SUCCESS - saved https://mk0foodfornetcoviwv0.kinstacdn.com/wp-content/uploads/exotic-fruits-from-around-the-world.png - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\d54ee7151f

C:\Users\saurabh\Anaconda3\lib\site-packages\PIL\Image.py:2929: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://www.mensjournal.com/wp-content/uploads/mf/_power_fruit_pomegranate_0.jpg?quality=86&strip=all - cannot identify image file <_io.BytesIO object at 0x000001FD7569F1D0>


C:\Users\saurabh\Anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://www.diabete.qc.ca/wp-content/uploads/2014/08/Les-fruits-768x768.png - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\e33a07e38c.jpg
SUCCESS - saved https://img.etimg.com/thumb/width-1200,height-900,imgsize-322201,resizemode-1,msid-75714887/magazines/panache/doctors-suggest-having-fruits-could-help-keep-coronavirus-at-bay.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\09d97eba7a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ01RgXcDAEeuaEoEN3SWnzBdaUGB2tTB2V6w&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\6217d19779.jpg
SUCCESS - saved https://www.luxuryvillasphuketthailand.com/wp-content/uploads/2018/01/Fruits-in-Phuket.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\dda58dbc26.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ6ikiQbCPFTkKjI_8w9JNDnzuGS4Ep2AB9tg&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\494ba42d5

SUCCESS - saved https://media.npr.org/assets/img/2019/07/17/gettyimages-688974837_custom-05a1fe7c8ce05a3a307dc7aa2af024713745dfe2.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\05a1fe7c8c.jpg
SUCCESS - saved https://i.pinimg.com/originals/bd/d5/e9/bdd5e983743da88cd9028b4b87b7dd73.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\aec3326141.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2A9Pch7uvfGdKgfIznLJs2dqlyZTVij4jVw&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\83d9bd3d47.jpg
SUCCESS - saved https://static.scientificamerican.com/sciam/cache/file/528E0B49-CDD0-42D4-B5BAA3EBAEC01AE6_source.jpg?w=590&h=800&2BDA5AE7-F98A-493B-A10B4C078F76B21B - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\fruits\8e61270ee8.jpg
SUCCESS - saved https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F37%2F2019%2F09%2F10184935%2Fred-green-apples-leaves-6fb7c0

SUCCESS - saved https://www.healthcareitnews.com/sites/hitn/files/Immunai%20AI%20Architecture%20%281%29%20%281%29.png - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\09e74f9058.jpg
SUCCESS - saved https://wp.technologyreview.com/wp-content/uploads/2018/11/algorithm2orange.1-9.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\6a1090c1ae.jpg
SUCCESS - saved https://www.mrdbourke.com/content/images/2020/07/ml-roadmap-blog-post-header.png - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\a6f46f6bc2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5wWLyJTkF70j5wUSDzkxWLPrmYJzI_CHTGQ&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\fdddda4449.jpg
SUCCESS - saved https://res.cloudinary.com/springboard-images/image/upload/q_auto,f_auto,fl_lossy/wordpress/2019/07/sb-blog-what-is-mle.png - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\8f71efc05e.jpg

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSxZZwCvSB5Xs37UOYvEKOZuGC8R3s40U3WDg&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\07a75b22ba.jpg
SUCCESS - saved https://s27389.pcdn.co/wp-content/uploads/2020/10/how-kubernetes-extends-machine-learning-ml.jpeg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\66b4894f81.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmX2Ela9pLW2dmQHa4Rra6xFdcVPyhk8kJHg&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\1b383a015f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQQdYjKvqIh60xvSFA41sdrmxntiPSwuWGl3g&usqp=CAU - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\f903377bb7.jpg
SUCCESS - saved https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/04_thumb.jpg - as C:\Users\saurabh\OneDrive\Desktop\New folder (3)\machine learning\203c39d593.jpg
SUCCESS - saved http

In [71]:
#scraping images of cars

# user can input a topic and a number
# download first n images from google image search

GOOGLE_IMAGE = \
    'https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&'


usr_agent = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive',
}

SAVE_FOLDER = 'images'

def main():
    if not os.path.exists(SAVE_FOLDER):
        os.mkdir(SAVE_FOLDER)
    download_images()


def download_images():
    data = input('What are you looking for? ')
    n_images = int(input('How many images do you want download? '))

    print('Start searching.....')
    searchurl = GOOGLE_IMAGE + 'q=' + data

    # request url, without usr_agent, the permission gets denied
    response = requests.get(searchurl, headers=usr_agent)

    # find all divs where class='rg_meta'
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.findAll('img', {'class': 'rg_i Q4LuWd'})

    # gathering requested number of list of image links with data-src attribute
    # continue the loop in case query fails for non-data-src attributes
    count = 0
    links = []
    for res in results:
        try:
            link = res['data-src']
            links.append(link)
            count += 1
            if (count >= n_images): break

        except KeyError:
            continue

    print(f'Downloading {len(links)} images....')

    # Access the data URI and download the image to a file
    for i, link in enumerate(links):
        response = requests.get(link)

        image_name = SAVE_FOLDER + '/' + data + str(i + 1) + '.jpg'
        with open(image_name, 'wb') as fh:
            fh.write(response.content)
            
        print('Done')


if __name__ == '__main__':
    main()

What are you looking for? cars
How many images do you want download? 100
Start searching.....
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page.Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [79]:
#connecting to the webdriver
driver = webdriver.Chrome(r"C:\Users\saurabh\Downloads\chromedriver_win32\chromedriver.exe")

In [80]:
#specifying the url of the webpage to be scraped
url="https://www.flipkart.com/"

In [81]:
#opening the webpage through our driver
driver.get(url)

In [82]:
# Taking input from user 
search_string = input("Input the string you want to search for:")

#generate a url from search string
url= driver.get("https://www.flipkart.com/search?q="+search_string+"&as=on&as-show=on&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&as-pos=4&as-type=RECENT&suggestionId=samsung+m31&requestId=5c8b7098-e268-4146-b626-2617dd872a64&as-searchtext=samsung%20")

Input the string you want to search for:samsung m31


In [84]:
#getting the list of url of samsung m31
Phone_URL=[]


phone=driver.find_elements_by_xpath("//div[@class='_2kHMtA']")
for i in phone:
    phone1=i.find_element_by_tag_name("a")
    Phone_URL.append(phone1.get_attribute('href'))

In [85]:
Phone_URL

['https://www.flipkart.com/samsung-galaxy-m31-ocean-blue-128-gb/p/itm1d7db8f4e2fec?pid=MOBFPNPSZYJXQZCM&lid=LSTMOBFPNPSZYJXQZCMEJJKKJ&marketplace=FLIPKART&srno=s_1_1&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&fm=organic&iid=defbae9d-b65e-41e7-9115-d627bb6c7a28.MOBFPNPSZYJXQZCM.SEARCH&ssid=5p3dg38bvk0000001612358914475&qH=72c0d4f69be69bf9',
 'https://www.flipkart.com/samsung-galaxy-m31-space-black-128-gb/p/itmeb29fb2c00580?pid=MOBFPNPS6QGTKBQB&lid=LSTMOBFPNPS6QGTKBQBH8ECKJ&marketplace=FLIPKART&srno=s_1_2&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&fm=organic&iid=defbae9d-b65e-41e7-9115-d627bb6c7a28.MOBFPNPS6QGTKBQB.SEARCH&ssid=5p3dg38bvk0000001612358914475&qH=72c0d4f69be69bf9',
 'https://www.flipkart.com/samsung-galaxy-m31-iceberg-blue-128-gb/p/itmed6043337ac0e?pid=MOBFYZ8Y9VJNACEM&lid=LSTMOBFYZ8Y9VJNACEMEBJFED&marketplace=FLIPKART&srno=s_1_3&otracker=AS_Query_OrganicAutoSu

In [86]:
#checking the length
len(Phone_URL)

24

In [169]:
#going to the url of first phone through our driver
driver.get(Phone_URL[0])

In [170]:
#creating empty lists

brand_name=[]
smartphone_name=[]
colour=[]
ram=[]
rom=[]
primary_camera=[]
secondary_camera=[]
display_size=[]
display_resolution=[]
processor=[]
processor_cores=[]
battery_capacity=[]
price=[]

In [171]:
#running a loop to extract all the required information from the Phone_URL
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
for i in tqdm(Phone_URL):
    driver.get(i)
    
    #try:
    #driver.find_element_by_link_text('Read More').click()
    #except NoSuchElementException:
        #driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > button').click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//button[text()="Read More"]'))).click()
    
    try:
        phone=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span').text
        smartphone_name.append(phone)
    except NoSuchElementException:
        smartphone_name.append("-")
    try:
        Colour=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(1) > table > tbody > tr:nth-child(4) > td.URwL2w.col.col-9-12 > ul > li').text
        colour.append(Colour)
    except NoSuchElementException:
        colour.append("-")
    try:
        RAM=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(4) > table > tbody > tr:nth-child(2) > td.URwL2w.col.col-9-12 > ul > li').text
        ram.append(RAM)
    except NoSuchElementException:
        ram.append("-")
        
    try:
        ROM=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(4) > table > tbody > tr:nth-child(1) > td.URwL2w.col.col-9-12 > ul > li').text
        rom.append(ROM)
    except NoSuchElementException:
        rom.append("-")
        
    try:
        Camera=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(5) > table > tbody > tr:nth-child(2) > td.URwL2w.col.col-9-12 > ul > li').text
        primary_camera.append(Camera)
    except NoSuchElementException:
        primary_camera.append("-")
        
    try:
        Camera_=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(5) > table > tbody > tr:nth-child(5) > td.URwL2w.col.col-9-12 > ul > li').text
        secondary_camera.append(Camera_)
    except NoSuchElementException:
        secondary_camera.append("-")
        
    try:
        Display=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(2) > table > tbody > tr:nth-child(1) > td.URwL2w.col.col-9-12 > ul > li').text
        display_size.append(Display)
    except NoSuchElementException:
        display_size.append("-")
        
    try:
        Display_=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(2) > table > tbody > tr:nth-child(2) > td.URwL2w.col.col-9-12 > ul > li').text
        display_resolution.append(Display_)
    except NoSuchElementException:
        display_resolution.append("-")
        
    try:
        Processor=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(3) > table > tbody > tr:nth-child(2) > td.URwL2w.col.col-9-12 > ul > li').text
        processor.append(Processor)
    except NoSuchElementException:
        processor.append("-")
        
    try:
        Cores=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(3) > table > tbody > tr:nth-child(3) > td.URwL2w.col.col-9-12 > ul > li').text
        processor_cores.append(Cores)
    except NoSuchElementException:
        processor_cores.append("-")
        
    try:
        Battery=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > div._1UhVsV > div:nth-child(9) > table > tbody > tr:nth-child(1) > td.URwL2w.col.col-9-12 > ul > li').text
        battery_capacity.append(Battery)
    except NoSuchElementException:
        battery_capacity.append("-")
        
    try:
        Price_=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div:nth-child(2) > div > div.dyC4hf > div.CEmiEU > div > div').text
        price.append(Price_)
    except NoSuchElementException:
        price.append("-")
        
    time.sleep(5)

100%|██████████| 24/24 [02:35<00:00,  6.48s/it]


In [172]:
smartphone_name

['Samsung Galaxy M31 (Ocean Blue, 128 GB)  (6 GB RAM)',
 'Samsung Galaxy M31 (Space Black, 128 GB)  (6 GB RAM)',
 'Samsung Galaxy M31 (Iceberg blue, 128 GB)  (6 GB RAM)',
 'Samsung Galaxy M31 (Space Black, 64 GB)  (6 GB RAM)',
 'Samsung Galaxy M31s (Mirage Blue, 128 GB)  (6 GB RAM)',
 'Samsung Galaxy M31s (Mirage Black, 128 GB)  (6 GB RAM)',
 'Samsung Galaxy M31 (Ocean Blue, 128 GB)  (8 GB RAM)',
 'Samsung Galaxy M31 (Ocean Blue, 64 GB)  (6 GB RAM)',
 'Samsung Galaxy M31 (Space Black, 128 GB)  (8 GB RAM)',
 'Samsung GALAXY M31S (Mirage Black, 128 GB)  (8 GB RAM)',
 'Samsung M31 Prime (Space Black, 128 GB)  (6 GB RAM)',
 'Samsung Galaxy A31 (Prism Crush Black, 128 GB)  (6 GB RAM)',
 'Samsung M31 Prime (Ocean Blue, 128 GB)  (6 GB RAM)',
 'Samsung M31s (Mirage Blue, 128 GB)  (8 GB RAM)',
 'Samsung M31 Prime (IceBerg Blue, 128 GB)  (6 GB RAM)',
 'Samsung M31 Prime (IceBerg Blue, 64 GB)  (6 GB RAM)',
 'Samsung M31 Prime (Ocean Blue, 64 GB)  (6 GB RAM)',
 'Samsung Galaxy A31 (Prism Crush Blu

In [173]:
colour

['Ocean Blue',
 'Space Black',
 'Iceberg blue',
 'Space Black',
 'Mirage Blue',
 '-',
 'Ocean Blue',
 'Ocean Blue',
 'Space Black',
 'Mirage Black',
 'Space Black',
 '-',
 'Ocean Blue',
 'Mirage Blue',
 'IceBerg Blue',
 '-',
 '-',
 '-',
 '-',
 'Blue',
 'Black',
 'Red',
 'Black',
 'Gold']

In [174]:
ram

['6 GB',
 '6 GB',
 '6 GB',
 '6 GB',
 '6 GB',
 '-',
 '8 GB',
 '6 GB',
 '8 GB',
 '8 GB',
 '6 GB',
 '-',
 '6 GB',
 '8 GB',
 '6 GB',
 '-',
 '-',
 '-',
 '-',
 '1 GB',
 '1 GB',
 '1 GB',
 '2 GB',
 '1 GB']

In [175]:
rom

['128 GB',
 '128 GB',
 '128 GB',
 '64 GB',
 '128 GB',
 '-',
 '128 GB',
 '64 GB',
 '128 GB',
 '128 GB',
 '128 GB',
 '-',
 '128 GB',
 '128 GB',
 '128 GB',
 '-',
 '-',
 '-',
 '-',
 '16 GB',
 '16 GB',
 '16 GB',
 '16 GB',
 '16 GB']

In [176]:
primary_camera

['64MP + 8MP + 5MP + 5MP',
 '64MP + 8MP + 5MP + 5MP',
 '64MP + 8MP + 5MP + 5MP',
 '64MP + 8MP + 5MP + 5MP',
 'Primary Camera',
 '-',
 '64MP + 8MP + 5MP + 5MP',
 '64MP + 8MP + 5MP + 5MP',
 '64MP + 8MP + 5MP + 5MP',
 'Primary Camera',
 'Primary Camera',
 '-',
 'Primary Camera',
 '64MP + 12MP',
 'Primary Camera',
 '-',
 '-',
 '-',
 '-',
 '8MP Rear Camera',
 '8MP Rear Camera',
 '8MP Rear Camera',
 '13MP Rear Camera',
 '8MP Rear Camera']

In [177]:
secondary_camera

['32MP Front Camera',
 '32MP Front Camera',
 '32MP Front Camera',
 '32MP Front Camera',
 '-',
 '-',
 '32MP Front Camera',
 '32MP Front Camera',
 '32MP Front Camera',
 '-',
 '-',
 '-',
 '-',
 '32MP Front Camera',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Yes',
 'Yes',
 'Yes',
 '5MP Front Camera',
 'Rear Flash']

In [178]:
display_size

['16.26 cm (6.4 inch)',
 '16.26 cm (6.4 inch)',
 '16.26 cm (6.4 inch)',
 '16.26 cm (6.4 inch)',
 '16.51 cm (6.5 inch)',
 '-',
 '16.26 cm (6.4 inch)',
 '16.26 cm (6.4 inch)',
 '16.26 cm (6.4 inch)',
 '16.51 cm (6.5 inch)',
 '16.26 cm (6.4 inch)',
 '-',
 '16.26 cm (6.4 inch)',
 '16.51 cm (6.5 inch)',
 '16.26 cm (6.4 inch)',
 '-',
 '-',
 '-',
 '-',
 '13.46 cm (5.3 inch)',
 '13.46 cm (5.3 inch)',
 '13.46 cm (5.3 inch)',
 '13.97 cm (5.5 inch)',
 '12.7 cm (5 inch)']

In [179]:
display_resolution

['2340 x 1080$$pixel',
 '2340 x 1080$$pixel',
 '2340 x 1080$$pixel',
 '2340 x 1080$$pixel',
 '1080 x 2400$$pixel',
 '-',
 '2340 x 1080$$pixel',
 '2340 x 1080$$pixel',
 '2340 x 1080$$pixel',
 '1080 x 2400$$pixel',
 '2340 x 1080$$pixel',
 '-',
 '2340 x 1080$$pixel',
 '1080 x 2400$$pixel',
 '2340 x 1080$$pixel',
 '-',
 '-',
 '-',
 '-',
 '1480 x 720$$pixel',
 '1480 x 720$$pixel',
 '1480 x 720$$pixel ',
 '1280 x 720 Pixels',
 '960 x 540 Pixels']

In [180]:
processor

['Samsung Exynos 9 Octa 9611',
 'Samsung Exynos 9 Octa 9611',
 'Samsung Exynos 9 Octa 9611',
 'Samsung Exynos 9 Octa 9611',
 'Octa Core',
 '-',
 'Samsung Exynos 9 Octa 9611',
 'Samsung Exynos 9 Octa 9611',
 'Samsung Exynos 9 Octa 9611',
 'Octa Core',
 'Deca Core',
 '-',
 'Deca Core',
 'Exynos 9611 octa core processor',
 'Deca Core',
 '-',
 '-',
 '-',
 '-',
 'MediaTek | MT6739WW quad core ',
 'MediaTek | MT6739WW quad core ',
 'MediaTek | MT6739WW quad core ',
 'Exynos Octa-Core 1.6GHz',
 'Qualcomm MSM8917']

In [181]:
processor_cores

['Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 '2.3 GHz',
 '-',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 '2.3 GHz',
 '1.7 GHz',
 '-',
 '1.7 GHz',
 'Octa Core',
 '1.7 GHz',
 '-',
 '-',
 '-',
 '-',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Quad Core']

In [182]:
battery_capacity

['6000 mAh',
 '6000 mAh',
 '6000 mAh',
 '6000 mAh',
 '-',
 '-',
 '6000 mAh',
 '6000 mAh',
 '6000 mAh',
 '-',
 '-',
 '-',
 '-',
 'MP3, M4A, 3GA, AAC, OGG, OGA, WAV, WMA, AMR, AWB, FLAC, MID, MIDI, XMF, MXMF, IMY, RTTTL, RTX, OTA',
 '-',
 '-',
 '-',
 '-',
 '-',
 'MP3, M4A, 3GA, AAC, OGG, WAV, AMR, AWB, FLAC, MID, XMF, MXMF, IMY, RTTTL, RTX, OTA',
 'MP3, M4A, 3GA, AAC, OGG, WAV, AMR, AWB, FLAC, MID, XMF, MXMF, IMY, RTTTL, RTX, OTA',
 'MP3, M4A, 3GA, AAC, OGG, WAV, AMR, AWB, FLAC, MID, XMF, MXMF, IMY, RTTTL, RTX, OTA',
 '3000 mAh',
 '2600 mAh']

In [183]:
price

['₹17,800',
 '₹17,499',
 '₹17,994',
 '₹15,950',
 '₹21,995',
 '₹21,990',
 '₹19,399',
 '₹15,990',
 '₹19,377',
 '₹22,490',
 '₹17,340',
 '₹17,999',
 '₹17,998',
 '₹21,880',
 '₹18,490',
 '₹18,699',
 '₹18,699',
 '₹17,999',
 '₹17,999',
 '₹5,382',
 '₹5,386',
 '₹5,499',
 '₹10,990',
 '₹7,700']

In [184]:
#extracting brand_name from smartphone_name
brand_name = [i.split(maxsplit=1)[0] for i in smartphone_name]

In [185]:
brand_name

['Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung',
 'Samsung']

In [186]:
#printing lengths
print(len(brand_name),len(smartphone_name),len(colour),len(ram),len(rom),len(primary_camera),len(secondary_camera),len(display_size),len(display_resolution),len(processor),len(processor_cores),len(battery_capacity),len(price),len(Phone_URL))

24 24 24 24 24 24 24 24 24 24 24 24 24 24


In [187]:
#creating a dataframe

Smartphone=pd.DataFrame({})
Smartphone['brand_name']=brand_name
Smartphone['smartphone_name']=smartphone_name
Smartphone['colour']=colour
Smartphone['Ram']=ram
Smartphone['Rom']=rom
Smartphone['primary_camera']=primary_camera
Smartphone['secondary_camera']=secondary_camera
Smartphone['display_size']=display_size
Smartphone['display_resolution']=display_resolution
Smartphone['processor']=processor
Smartphone['processor_cores']=processor_cores
Smartphone['battery_capacity']=battery_capacity
Smartphone['price']=price
Smartphone['Phone_URL']=Phone_URL

In [188]:
Smartphone

,brand_name,smartphone_name,colour,Ram,Rom,primary_camera,secondary_camera,display_size,display_resolution,processor,processor_cores,battery_capacity,price,Phone_URL
0,Samsung,"Samsung Galaxy M31 (Ocean Blue, 128 GB) (6 GB...",Ocean Blue,6 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹17,800",https://www.flipkart.com/samsung-galaxy-m31-oc...
1,Samsung,"Samsung Galaxy M31 (Space Black, 128 GB) (6 G...",Space Black,6 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹17,499",https://www.flipkart.com/samsung-galaxy-m31-sp...
2,Samsung,"Samsung Galaxy M31 (Iceberg blue, 128 GB) (6 ...",Iceberg blue,6 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹17,994",https://www.flipkart.com/samsung-galaxy-m31-ic...
3,Samsung,"Samsung Galaxy M31 (Space Black, 64 GB) (6 GB...",Space Black,6 GB,64 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹15,950",https://www.flipkart.com/samsung-galaxy-m31-sp...
4,Samsung,"Samsung Galaxy M31s (Mirage Blue, 128 GB) (6 ...",Mirage Blue,6 GB,128 GB,Primary Camera,-,16.51 cm (6.5 inch),1080 x 2400$$pixel,Octa Core,2.3 GHz,-,"₹21,995",https://www.flipkart.com/samsung-galaxy-m31s-m...
5,Samsung,"Samsung Galaxy M31s (Mirage Black, 128 GB) (6...",-,-,-,-,-,-,-,-,-,-,"₹21,990",https://www.flipkart.com/samsung-galaxy-m31s-m...
6,Samsung,"Samsung Galaxy M31 (Ocean Blue, 128 GB) (8 GB...",Ocean Blue,8 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹19,399",https://www.flipkart.com/samsung-galaxy-m31-oc...
7,Samsung,"Samsung Galaxy M31 (Ocean Blue, 64 GB) (6 GB ...",Ocean Blue,6 GB,64 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹15,990",https://www.flipkart.com/samsung-galaxy-m31-oc...
8,Samsung,"Samsung Galaxy M31 (Space Black, 128 GB) (8 G...",Space Black,8 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹19,377",https://www.flipkart.com/samsung-galaxy-m31-sp...
9,Samsung,"Samsung GALAXY M31S (Mirage Black, 128 GB) (8...",Mirage Black,8 GB,128 GB,Primary Camera,-,16.51 cm (6.5 inch),1080 x 2400$$pixel,Octa Core,2.3 GHz,-,"₹22,490",https://www.flipkart.com/samsung-galaxy-m31s-m...


In [189]:
#converting into csv format
Smartphone.to_csv("Smartphone.csv")

In [190]:
#closing the driver
driver.close()

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [191]:
driver = webdriver.Chrome(r"C:\Users\saurabh\Downloads\chromedriver_win32\chromedriver.exe")

In [192]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re

sentence = "Bengaluru"
url = 'https://www.google.com/maps/search/{}'.format(sentence)

driver.get(url)
r = re.compile(r'center=(-?\d+\.\d+)%2C(-?\d+\.\d+)')
lat_long = r.findall(driver.page_source)


print(lat_long)

driver.quit()

[('12.9539974', '77.6309395'), ('12.9539974', '77.6309395')]


In [193]:
coordinates=lat_long[0:1]
coordinates

[('12.9539974', '77.6309395')]

### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in

In [194]:
#connecting to the webdriver
driver = webdriver.Chrome(r"C:\Users\saurabh\Downloads\chromedriver_win32\chromedriver.exe")

In [195]:
#specifying the url of the webpage to be scraped
url = "https://trak.in/"

In [196]:
#opening the url through our driver
driver.get(url)

In [197]:
#clicking on funding deals
driver.find_element_by_xpath('//*[@id="menu-item-51510"]/a').click()

In [218]:
#creating empty lists

Date=[]
Startup_name=[]
Vertical=[]
Sub_vertical=[]
Location=[]
Investors_name=[]
Investment_type=[]
Amount=[]

In [219]:
#finding all the date tags between July 20 - September 20
date = driver.find_elements_by_xpath("//td[@class='column-2']")[34:67]
date[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="6efabc6d-747a-49bb-ae6e-34ff69489327")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="c62fac2c-e0ce-4c86-bd76-ba13640f4834")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="e00df8b6-89ea-421a-8345-6690a71de393")>]

In [220]:
#scraping date from date tags
for i in date:
    dates=i.text.replace("\n"," ")
    Date.append(dates)
Date

['08/09/2020',
 '12/09/2020',
 '09/09/2020',
 '02/09/2020',
 '09/09/2020',
 '07/09/2020',
 '07/09/2020',
 '31/08/2020',
 '31/08/2020',
 '03/09/2020',
 '08/09/2020',
 '15/08/2020',
 '13/08/2020',
 '13/08/2020',
 '04/08/2020',
 '11/08/2020',
 '12/08/2020',
 '14/08/2020',
 '10/08/2020',
 '07/08/2020',
 '13/08/2020',
 '03/08/2020',
 '04/08/2020',
 '15/07/2020',
 '16/07/2020',
 '16/07/2020',
 '14/07/2020',
 '13/07/2020',
 '09/07/2020',
 '10/07/2020',
 '15/07/2020',
 '14/07/2020',
 '13/07/2020']

In [221]:
#finding all the startup_name tags between July 20 - September 20
startup = driver.find_elements_by_xpath("//td[@class='column-3']/a")[34:67]
startup[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="44efae18-cab7-45a3-b91d-8f54af3af770")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="e7897280-9eb9-4830-96c1-0909fba49950")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="9ee534cb-e15f-4b4c-97ae-21f395f2347a")>]

In [222]:
#scraping startup_name from tags
for i in startup:
    startups=i.text.replace("\n"," ")
    Startup_name.append(startups)
Startup_name

['Byju’s',
 'mCaffeine',
 'Qshala',
 'Winzo',
 'Hippo Video',
 'Melorra',
 '1mg',
 'mfine',
 'Apna',
 'Railofy',
 'Cell Propulsion',
 'Practo',
 'Medlife',
 'HungerBox',
 'Dunzo',
 'Terra.do',
 'Classplus',
 'Niyo',
 'ZestMoney',
 'FreshToHome',
 'Eduvanz',
 'CrowdPouch',
 'DrinkPrime',
 'Flipkart',
 'Vedantu',
 'Crio',
 'goDutch',
 'Mystifly',
 'JetSynthesys',
 'gigIndia',
 'PumPumPum',
 'FLYX',
 'Open Appliances Pvt. Ltd.']

In [223]:
#finding all the Vertical tags between July 20 - September 20
vertical = driver.find_elements_by_xpath("//td[@class='column-4']")[34:67]
vertical[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="47b1ff26-402f-459f-9591-bc17d1ab6aa6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="4512cca6-54ea-4bb5-bca5-c10ae933201f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="fb1698d9-3cfc-4114-b4e4-ad802e020757")>]

In [224]:
#scraping Vertical from tags
for i in vertical:
    verticals=i.text.replace("\n"," ")
    Vertical.append(verticals)
Vertical

['EduTech',
 'Personal Care',
 'EduTech',
 'Online Gaming',
 'Video Customer Experience(CX) Platform',
 'E-commerce',
 'E-commerce',
 'HealthTech',
 'Human Resources',
 'Transportation',
 'Automobile',
 'HealthTech',
 'E-commerce',
 'FoodTech',
 'Hyper-local Logistics',
 'EduTech',
 'EduTech',
 'FinTech',
 'FinTech',
 'E-commerce',
 'FinTech',
 'FinTech',
 'Water Purification',
 'E-commerce',
 'EduTech',
 'EduTech',
 'FinTech',
 'Airfare Marketplace',
 'Gaming and Entertainment',
 'Marketplace',
 'Automotive Rental',
 'OTT Player',
 'Information Technology']

In [225]:
#finding all the Sub_vertical tags between July 20 - September 20
sub_vertical = driver.find_elements_by_xpath("//td[@class='column-5']")[34:67]
sub_vertical[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="68afae78-0cb6-42eb-a074-02f70a5fa90d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="5632c736-902e-40c6-99fb-0db5cc3b360d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="71f57055-0490-4fff-9ecd-eb507ca07702")>]

In [226]:
#scraping Sub_vertical from tags
for i in sub_vertical:
    sub_verticals=i.text.replace("\n"," ")
    Sub_vertical.append(sub_verticals)
Sub_vertical

['Online Tutoring',
 'Skincare & Haircare',
 'Online Curiosity Platform for Kids',
 'Online Gaming',
 'Video Customer Experience(CX) Platform',
 'Online Jewelry Store',
 'Online Pharmacy',
 'On-Demand Healthcare Services',
 'Recruitment Platform',
 'WL & RAC protection platform',
 'Electric Mobility Solutions',
 'Health care and Wellness',
 'Online Pharmacy',
 'Online Food Delivery Service',
 'Online Delivery Services',
 'Online Climate School, E-learning',
 'E-learning, Online Tutoring',
 'Financial Services',
 'Financial Services',
 'Food Delivery',
 'Financial Services',
 'Financial Services',
 'Water Purification',
 'E-commerce',
 'Online Tutoring',
 'Learning Platform for Developers',
 'Group Payments',
 'Ticketing, Airline Retailing, and Post-Ticketing Services',
 'Gaming and Entertainment',
 'Crowd Sourcing, Freelance',
 'Used Car-leasing platform',
 'Streaming Social Network',
 'Internet-of-Things Security Solutions']

In [227]:
#finding all the location tags between July 20 - September 20
location = driver.find_elements_by_xpath("//td[@class='column-6']")[34:67]
location[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="94272e21-a072-40bb-a91c-e085683b69c8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="2362c846-4446-4cb0-8dbc-545cb73bc360")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="8101f392-3500-4fa4-9c22-cca5d4b88635")>]

In [228]:
#scraping Location from tags
for i in location:
    locations=i.text.replace("\n"," ")
    Location.append(locations)
Location

['Bangalore',
 'Mumbai',
 'Bangalore',
 'New Delhi',
 'Newark, Delaware, United States of Amercia',
 'Bangalore',
 'Gurgaon',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Stanford, California,',
 'Noida',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Singapore and Bangalore',
 'Pune',
 'Pune',
 'Gurgaon',
 'New York and Delhi',
 'Bangalore']

In [229]:
#finding all the investors_name tags between July 20 - September 20
investors_name = driver.find_elements_by_xpath("//td[@class='column-7']")[34:67]
investors_name[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="a41c68d4-6f30-4d0c-ae97-a7b3098f590c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="1eb62d4c-df38-44da-a433-5ebbb73b8df8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="27911501-8770-40d5-b342-a2630a51025d")>]

In [230]:
#scraping Investors_name from tags
for i in investors_name:
    investors_names=i.text.replace("\n"," ")
    Investors_name.append(investors_names)
Investors_name

['Silver Lake, Tiger Global, General Atlantic and Owl Ventures',
 'Amicus Capital Private Equity I LLP, Amicus Capital Partners India Fund I and RP-SG Venture Fund 1',
 'Rainmatter Capital',
 'Kalaari Capital Partners, IndigoEdge Management Consultancy',
 'Alpha Wave Incubation, Exfinity Venture Partners and existing backers.',
 'Shadow Holdings, Lightbox.',
 'Gaja Capital, Tata Capital, Partners Group',
 'Caretech Pte Inc',
 'Lightspeed India and Sequoia Capital India',
 'Chiratae Ventures',
 'growX Ventures and Micelio',
 'A1A Company',
 'Prasid Uno Family Trust and SC Credit Fund',
 'One97, Sabre Partners Trust, Pratithi Investment Trust, and Srihari Kumar',
 'Existing Backers',
 'Stanford Angels and Entrepreneurs (India), BEENEXT Emerging Asia, Rainmatter Capital',
 'Falcon Edge',
 'Niyo Solutions Inc.',
 'Primrose Hills Ventures',
 'Ascent Capital',
 'Sequoia India, Unitus',
 'Elina Investments Pvt. Ltd',
 'Sequoia Surge, ON Mauritius',
 'Walmart Inc',
 'Coatue Management',
 '021 

In [231]:
#finding all the investment_type tags between July 20 - September 20
investment_type = driver.find_elements_by_xpath("//td[@class='column-8']")[34:67]
investment_type[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="3d677732-a74f-431b-8edf-2647d666ad1d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="4a3c5fd3-0628-4f45-95c5-2389e11fd808")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="48aef6e9-ec45-43db-b8fe-277c9e2ce7d3")>]

In [232]:
#scraping Investment_type from tags
for i in investment_type:
    investment_types=i.text.replace("\n"," ")
    Investment_type.append(investment_types)
Investment_type

['Private Equity',
 'Series B',
 'Angel',
 'Series B',
 'Series A',
 'Debt Financing',
 'In Progress',
 'Series B',
 'Series A',
 'Seed',
 'pre-Series A',
 'Series F',
 '',
 'Series D1',
 'In Progress',
 'Seed',
 'In Progress',
 '',
 '',
 'Venture',
 'Series A',
 'Angel',
 'Pre-Series A',
 'M&A',
 'Series D',
 'pre-Series A',
 'Seed',
 'pre-Series B',
 'Venture-Series Unknown',
 'pre-Series A',
 'Seed',
 'pre-Seed',
 'Venture-Series Unknown']

In [245]:
#finding all the amount tags between July 20 - September 20
amount = driver.find_elements_by_xpath("//td[@class='column-9']")[34:67]
amount[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="0c7a6bc4-035f-4747-b61c-c38115a88667")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="b4789240-435a-4b9b-b459-b3b45e41b0f1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12440e94b6be5c6d722c0220a5ec7f60", element="08348d4e-fd18-4e85-bdf4-e16315ce607a")>]

In [246]:
#scraping Amount from tags
for i in amount:
    amounts=i.text.replace("\n"," ")
    Amount.append(amounts)
Amount

['500,000,000',
 '3,000,000',
 '370,000',
 '15,500,000',
 '4,500,000',
 'upto 8,900,000',
 '100,000,000',
 '5,400,000',
 '8,000,000',
 '950,000',
 'NA',
 '32,000,000',
 '23,000,000',
 '1,560,000',
 '30,000,000',
 '1,400,000',
 'upto 15,000,000',
 '6,000,000',
 '10,670,000',
 '16,200,000',
 '5,000,000',
 'NA',
 '2,880,000',
 '1,200,000,000',
 '100,000,000',
 '934,160',
 '1,700,000',
 '3,300,000',
 '400,000',
 '974,200',
 '292,800',
 '200,000',
 '500,000']

In [247]:
#printing lengths
print(len(Date),len(Startup_name),len(Vertical),len(Sub_vertical),len(Location),len(Investors_name),len(Investment_type),len(Amount))

33 33 33 33 33 33 33 33


In [249]:
#creating a dataframe

Funding_deals=pd.DataFrame({})
Funding_deals['Date']=Date
Funding_deals['Startup_name']=Startup_name
Funding_deals['Vertical']=Vertical
Funding_deals['Sub_vertical']=Sub_vertical
Funding_deals['Location']=Location
Funding_deals['Investors_name']=Investors_name
Funding_deals['Investment_type']=Investment_type
Funding_deals['Amount']=Amount

In [250]:
Funding_deals

,Date,Startup_name,Vertical,Sub_vertical,Location,Investors_name,Investment_type,Amount
0,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
1,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
2,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
3,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
4,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
5,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
6,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
7,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
8,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
9,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


In [251]:
#converting into csv format
Funding_deals.to_csv("Funding_deals.csv")

In [252]:
#closing the driver
driver.close()

### 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in. 

In [371]:
#connecting to the webdriver
driver = webdriver.Chrome(r"C:\Users\saurabh\Downloads\chromedriver_win32\chromedriver.exe")

In [372]:
#specifying the url of the webpage to be scraped
url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [373]:
#opening the url through our driver
driver.get(url)

In [264]:
#finding all the seller tags
seller = driver.find_elements_by_class_name("smmerchant")
seller[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="9134b591-0031-4cb2-a974-ced2d62d6b39")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="a34bb9ac-7075-472f-b211-8773fb9c6e07")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="e17f0c68-355c-4c7b-916c-d2d35158e3f6")>]

In [265]:
#scraping Seller from seller tags
Seller=[]
for i in seller:
    sellers=i.text.replace("\n"," ")
    Seller.append(sellers)
Seller

['N/A',
 'amazon',
 'Tatacliq',
 'amazon',
 'flipkart',
 'flipkart',
 'amazon',
 'N/A',
 'amazon',
 'amazon']

In [266]:
#finding all the price tags
price = driver.find_elements_by_class_name("smprice")
price[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="11230b81-5f70-404f-9bc7-0b1bf4dac90d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="cfde6849-cc71-4492-9fc7-c3ff66e42da3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="8c80d1e8-213a-4b03-be58-52eb1d6dce77")>]

In [267]:
#scraping price from price tags
PRICE=[]
for i in price:
    prices=i.text.replace("\n"," ")
    PRICE.append(prices)
PRICE

['₹379990',
 '₹489909',
 '₹117790',
 '₹164990',
 '₹79990',
 '₹79990',
 '₹146000',
 '₹349990',
 '₹268990',
 '₹61990']

In [271]:
#finding all the os tags
os = driver.find_elements_by_xpath("//*[contains(text(), 'OS')]")
os[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="7e16b5de-bb3c-4da1-a7d0-d95e29af19c5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="7f0266f3-815d-4d32-87d4-f801732e2611")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="acec5f3b-43dc-42a1-8e26-d331e154b3fe")>]

In [273]:
#scraping os from tags
OS=[]
for i in os:
    os_=i.text.replace("\n"," ")
    OS.append(os_)
OS

['',
 'OS WINDOWS 10 PRO',
 'OS',
 'OS WINDOWS 10 PRO',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME 64 BIT',
 'OS',
 'OS WINDOWS 10',
 'OS',
 'OS WINDOWS 10 HOME PLUS',
 'OS']

In [275]:
#slicing only odd elements from list
OS_ = OS[1::2]
OS_

['OS WINDOWS 10 PRO',
 'OS WINDOWS 10 PRO',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME 64 BIT',
 'OS WINDOWS 10',
 'OS WINDOWS 10 HOME PLUS']

In [289]:
Operating_system = list(map(lambda x: x.replace('OS ',''),OS_))
Operating_system

['WINDOWS 10 PRO',
 'WINDOWS 10 PRO',
 'WINDOWS 10 HOME',
 'WINDOWS 10 HOME',
 'WINDOWS 10 HOME',
 'WINDOWS 10 HOME',
 'WINDOWS 10 HOME',
 'WINDOWS 10 HOME 64 BIT',
 'WINDOWS 10',
 'WINDOWS 10 HOME PLUS']

In [290]:
#finding all the display tags
display = driver.find_elements_by_xpath("//*[contains(text(), 'Display')]")
display[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="34ba396b-0a50-4263-89e5-63572481976f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="64d91a2f-86cd-48be-b536-fb00e1da8c84")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="511a325e-a769-478a-a649-65dd94549bbf")>]

In [294]:
#scraping os from tags
Display=[]
for i in display:
    display_=i.text.replace("\n"," ")
    Display.append(display_)
Display

['DISPLAY 17.3" (UHD 3840X2160)',
 'Display',
 'DISPLAY 17.3" (FHD (1920 X 1080))',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display',
 'DISPLAY 14" (1920 X 1080)',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display could use more colour',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display',
 'DISPLAY 18.4" (1920 X 1080)',
 'Display',
 'DISPLAY 15.6" (3840 X 1100)',
 'Display',
 'DISPLAY 15.6 MP | NA',
 'Display',
 '']

In [295]:
#poping unwanted elements
indices = 1,3,5,7,9,11,12,14,16,18,20,21
Display = [i for j, i in enumerate(Display) if j not in indices]
Display

['DISPLAY 17.3" (UHD 3840X2160)',
 'DISPLAY 17.3" (FHD (1920 X 1080))',
 'DISPLAY 15.6" (1920 X 1080)',
 'DISPLAY 14" (1920 X 1080)',
 'DISPLAY 15.6" (1920 X 1080)',
 'DISPLAY 15.6" (1920 X 1080)',
 'DISPLAY 15.6" (1920 X 1080)',
 'DISPLAY 18.4" (1920 X 1080)',
 'DISPLAY 15.6" (3840 X 1100)',
 'DISPLAY 15.6 MP | NA']

In [296]:
Display = list(map(lambda x: x.replace('DISPLAY ',''),Display))
Display

['17.3" (UHD 3840X2160)',
 '17.3" (FHD (1920 X 1080))',
 '15.6" (1920 X 1080)',
 '14" (1920 X 1080)',
 '15.6" (1920 X 1080)',
 '15.6" (1920 X 1080)',
 '15.6" (1920 X 1080)',
 '18.4" (1920 X 1080)',
 '15.6" (3840 X 1100)',
 '15.6 MP | NA']

In [297]:
#finding all the processor tags
processor = driver.find_elements_by_xpath("//*[contains(text(), 'Processor')]")
processor[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="66e486dd-205c-4c60-a5eb-242d972cb00e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="89cba721-f3ea-472f-ab6d-982c57ac063e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="82ae3e23-8e86-4a2f-9580-4c7977f6e015")>]

In [298]:
#scraping processor from tags
Processor=[]
for i in processor:
    processor_=i.text.replace("\n"," ")
    Processor.append(processor_)
Processor

['PROCESSOR INTEL 9TH GEN CORE I9-9900K | 5000 MHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR INTEL 9TH GEN CORE I9-9900K | 5000 MHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR INTEL I7-10750H 10TH GEN | 1.6GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR 9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR 9TH GEN INTEL CORE I7-8750H | 2.2 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR INTEL CORE I7 10TH GEN 10875H | NA',
 'Processor',
 'Graphics Processor',
 'PROCESSOR 8TH GENERATION INTEL® CORE™ I7-8750H | NA',
 'Processor',
 'Graphics Processor',
 'Best Laptops with Intel Core i5 Processor']

In [302]:
#keeping required elements
indices=[0,3,6,9,12,15,18,21,24,27]
Processor = [ Processor[index] for index in indices ]
Processor

['PROCESSOR INTEL 9TH GEN CORE I9-9900K | 5000 MHZ',
 'PROCESSOR INTEL 9TH GEN CORE I9-9900K | 5000 MHZ',
 'PROCESSOR INTEL I7-10750H 10TH GEN | 1.6GHZ',
 'PROCESSOR AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ',
 'PROCESSOR 9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ',
 'PROCESSOR AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ',
 'PROCESSOR 9TH GEN INTEL CORE I7-8750H | 2.2 GHZ',
 'PROCESSOR INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ',
 'PROCESSOR INTEL CORE I7 10TH GEN 10875H | NA',
 'PROCESSOR 8TH GENERATION INTEL® CORE™ I7-8750H | NA']

In [303]:
Processor = list(map(lambda x: x.replace('PROCESSOR ',''),Processor))
Processor

['INTEL 9TH GEN CORE I9-9900K | 5000 MHZ',
 'INTEL 9TH GEN CORE I9-9900K | 5000 MHZ',
 'INTEL I7-10750H 10TH GEN | 1.6GHZ',
 'AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ',
 '9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ',
 'AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ',
 '9TH GEN INTEL CORE I7-8750H | 2.2 GHZ',
 'INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ',
 'INTEL CORE I7 10TH GEN 10875H | NA',
 '8TH GENERATION INTEL® CORE™ I7-8750H | NA']

In [304]:
#finding all the Memory tags
memory = driver.find_elements_by_xpath("//*[contains(text(), 'Memory')]")
memory[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="eb8ed93e-d41b-4f18-85d8-221b210b4b08")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="582c6826-579d-44f4-96af-9542558d7df9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="75dade24-b982-416e-af76-ea3a8125ee43")>]

In [306]:
#scraping Memory from tags
Memory=[]
for i in memory:
    memory_=i.text.replace("\n"," ")
    Memory.append(memory_)
Memory

['MEMORY 1 TB HDD/64 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB PCIE SSD/32GB DDR4',
 'Memory',
 'MEMORY 512 GB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB SSD/8GB DDR4',
 'Memory',
 'MEMORY 512 GB SSD/16GB DDR4',
 'Memory',
 'MEMORY 512GB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1.5 TB SATA/64GB DDR4',
 'Memory',
 'MEMORY 512 GB SSD/4 GBGB DDR4',
 'Memory',
 'MEMORY 256 + 1 TB SSD + SATA/16GB DDR4',
 'Memory']

In [314]:
#keeping required elements

indices=[0,2,4,6,8,10,12,14,16,18]
Memory = [ Memory[index] for index in indices ]
Memory

['MEMORY 1 TB HDD/64 GBGB DDR4',
 'MEMORY 1 TB PCIE SSD/32GB DDR4',
 'MEMORY 512 GB SSD/16 GBGB DDR4',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'MEMORY 1 TB SSD/8GB DDR4',
 'MEMORY 512 GB SSD/16GB DDR4',
 'MEMORY 512GB SSD/16 GBGB DDR4',
 'MEMORY 1.5 TB SATA/64GB DDR4',
 'MEMORY 512 GB SSD/4 GBGB DDR4',
 'MEMORY 256 + 1 TB SSD + SATA/16GB DDR4']

In [315]:
Memory = list(map(lambda x: x.replace('MEMORY ',''),Memory))
Memory

['1 TB HDD/64 GBGB DDR4',
 '1 TB PCIE SSD/32GB DDR4',
 '512 GB SSD/16 GBGB DDR4',
 '1 TB SSD/16 GBGB DDR4',
 '1 TB SSD/8GB DDR4',
 '512 GB SSD/16GB DDR4',
 '512GB SSD/16 GBGB DDR4',
 '1.5 TB SATA/64GB DDR4',
 '512 GB SSD/4 GBGB DDR4',
 '256 + 1 TB SSD + SATA/16GB DDR4']

In [322]:
#finding all the Weight tags
weight = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
weight[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="a23bf756-156c-453e-9ad2-480e61d2ae2a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="6f4da642-9851-4f27-9bf3-cacde70038f1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="6a982d59-9a96-453c-82d5-48924e10b17e")>]

In [323]:
#scraping Weight from tags
Weight=[]
for i in weight:
    weight_=i.text.replace("\n"," ")
    Weight.append(weight_)
Weight

['4.2 kg',
 '3.87 Kg',
 '5.40',
 '1.65',
 '2.3',
 '2.2',
 '2.1',
 '5.5',
 '2.4',
 '2.53']

In [332]:
#finding all the dimension tags
graphics = driver.find_elements_by_xpath("//*[contains(text(), 'TX')]")
graphics[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="2dd4c36c-b61d-49d4-8b5d-c182a7853e05")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="6fc6cf8b-2ba2-4904-b441-acd73a04a538")>,
 <selenium.webdriver.remote.webelement.WebElement (session="aecf65ebe92ba67e47aa0ced53a29fd8", element="12768e47-d394-4a18-ba5e-6a8e7cd8fd17")>]

In [333]:
#scraping dimension from tags
Graphics=[]
for i in graphics:
    graphics_=i.text.replace("\n"," ")
    Graphics.append(graphics_)
Graphics

['',
 'Sporting a desktop graphics Intel Core i9-9900K coupled with an RTX 2080, the MSI GT76 TITAN 9SG has the same configuration as the Alienware Area-51m. Both machines use massive power bricks which when plugged in trigger an automatic factory overclock of 32-33% for the processors. Even the graphics cards are the same RTX 2080 but MSI has a slightly higher base clock of 1575 MHz whereas the Alienware unit has that set to 1515 MHz. Boost clocks are also different with the MSI unit clocked at 1755 MHz and the Alienware at 1720 MHz. So performance wise, these two are nearly similar until you consider the next two important things in a computer, the RAM and the SSD boot drives. MSI’s gaming laptop has RAM clocked at 2666 MHz and uses two NVMe SSDs in RAID 0 for the boot drive.',
 'NVIDIA GeForce RTX 2080',
 'The Alienware Area-51m was certainly the first of its kind to have the desktop grade Intel Core i9-9900K coupled with NVIDIA’s RTX 2080. However, as time goes by competitors broug

In [334]:
#keeping required elements

indices=[2,4,7,9,11,13,14,16,18,20]
Graphics = [ Graphics[index] for index in indices ]
Graphics

['NVIDIA GeForce RTX 2080',
 'NVIDIA GeForce RTX 2080',
 'Nvidia GeForce GTX 1650Ti',
 'NVIDIA GeForce RTX 2060',
 'NVIDIA® GeForce RTX™ 2060',
 'NVIDIA Geforce GTX 1660 Ti',
 'NVIDIA® GeForce RTX™ 2070 (Max-Q)',
 'Dual GTX1070',
 'NVIDIA GeForce RTX 2070 Max-Q',
 'NVIDIA® GeForce® GTX']

In [374]:
#finding all the description tags
description = driver.find_elements_by_xpath("//div[@class='Section-center']/p")
description[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="f4f1074a2471c2390df4a88c6ccd997c", element="4df62c9e-a389-4762-80d2-c66f1378173a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4f1074a2471c2390df4a88c6ccd997c", element="1684a9cb-5539-4f23-ba0e-851a70221dd4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4f1074a2471c2390df4a88c6ccd997c", element="29bac300-1a8f-4c85-a876-00e651ff9cfe")>]

In [375]:
#scraping description from tags
Description=[]
for i in description:
    description_=i.text.replace("\n"," ")
    Description.append(description_)
Description

['Sporting a desktop graphics Intel Core i9-9900K coupled with an RTX 2080, the MSI GT76 TITAN 9SG has the same configuration as the Alienware Area-51m. Both machines use massive power bricks which when plugged in trigger an automatic factory overclock of 32-33% for the processors. Even the graphics cards are the same RTX 2080 but MSI has a slightly higher base clock of 1575 MHz whereas the Alienware unit has that set to 1515 MHz. Boost clocks are also different with the MSI unit clocked at 1755 MHz and the Alienware at 1720 MHz. So performance wise, these two are nearly similar until you consider the next two important things in a computer, the RAM and the SSD boot drives. MSI’s gaming laptop has RAM clocked at 2666 MHz and uses two NVMe SSDs in RAID 0 for the boot drive.',
 'The Alienware Area-51m was certainly the first of its kind to have the desktop grade Intel Core i9-9900K coupled with NVIDIA’s RTX 2080. However, as time goes by competitors brought similar configurations with sl

In [377]:
#finding all the laptop_name tags
laptop = driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
laptop[0:3]

[<selenium.webdriver.remote.webelement.WebElement (session="f4f1074a2471c2390df4a88c6ccd997c", element="c0e5b083-671e-41ff-a3b4-c1fc593720df")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4f1074a2471c2390df4a88c6ccd997c", element="1bc5d2d9-1ff1-485b-910a-4d6a6a016bcb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f4f1074a2471c2390df4a88c6ccd997c", element="c41e8705-d701-458b-9fe0-596e66f8822c")>]

In [378]:
#scraping laptop_name from tags
Laptop_name=[]
for i in laptop:
    laptop_=i.text.replace("\n"," ")
    Laptop_name.append(laptop_)
Laptop_name

['MSI GT76 TITAN DT 9SG',
 'ALIENWARE 17 AREA-51M',
 'HP OMEN 15 2020',
 'ASUS ZEPHYRUS G14',
 'LENOVO LEGION Y540',
 'ASUS ROG ZEPHYRUS G GA502',
 'ASUS ROG ZEPHYRUS S GX531',
 'MSI GT83VR 7RE TITAN SLI',
 'ASUS ROG ZEPHYRUS DUO 15',
 'DELL G3']

In [335]:
#getting the list of url of laptops
Laptop_URL=[]


laptop=driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']")
for i in laptop:
    laptop1=i.find_element_by_tag_name("a")
    Laptop_URL.append(laptop1.get_attribute('href'))

In [336]:
Laptop_URL

['https://www.digit.in/laptops/msi-gt76-titan-dt-9sg-price-193526.html',
 'https://www.digit.in/laptops/alienware-area-51m-core-i9-9900k-rtx-2080-price-193540.html',
 'https://www.digit.in/laptops/hp-omen-15-2020-price-207633.html',
 'https://www.digit.in/laptops/asus-rog-zephyrus-g14-price-209173.html',
 'https://www.digit.in/laptops/lenovo-legion-y540-price-184975.html',
 'https://www.digit.in/laptops/asus-rog-zephyrus-g-ga502-price-187290.html',
 'https://www.digit.in/laptops/asus-rog-zephyrus-s-gx531-price-183275.html',
 'https://www.digit.in/laptops/msi-gt83vr-7re-titan-sli-price-55229.html',
 'https://www.digit.in/laptops/asus-rog-zephyrus-duo-15-price-215799.html',
 'https://www.digit.in/laptops/dell-g3-15-price-172543.html']

In [379]:
#going to the url of first laptop through our driver
driver.get(Laptop_URL[0])

In [380]:
#creating empty lists
Market_status=[]
Release_date=[]
Official_website=[]
Display_rating=[]
CPU_rating=[]
GPU_rating=[]
RAM_rating=[]
Storage_rating=[]
Overall_rating=[]
Availibility=[]
Digit_rating=[]
User_rating=[]
Pros=[]
Cons=[]

In [381]:
#running a loop to extract all the required information from the Laptop_URL

for i in tqdm(Laptop_URL):
    driver.get(i)
    
    try:
        market=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.status-bar.only-desktop > div:nth-child(1) > b').text
        Market_status.append(market)
    except NoSuchElementException:
        Market_status.append("-")
    try:
        release=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.status-bar.only-desktop > div:nth-child(2) > b').text
        Release_date.append(release)
    except NoSuchElementException:
        Release_date.append("-")
        
    try:
        website=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.status-bar.only-desktop > div:nth-child(3) > b').text
        Official_website.append(website)
    except NoSuchElementException:
        Official_website.append("-")
        
    try:
        display=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(2) > div > span').text
        Display_rating.append(display)
    except NoSuchElementException:
        Display_rating.append("-")
        
    try:
        cpu=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(3) > div > span').text
        CPU_rating.append(cpu)
    except NoSuchElementException:
        CPU_rating.append("-")
        
    try:
        gpu=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(4) > div > span').text
        GPU_rating.append(gpu)
    except NoSuchElementException:
        GPU_rating.append("-")
        
    try:
        ram=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(5) > div > span').text
        RAM_rating.append(ram)
    except NoSuchElementException:
        RAM_rating.append("-")
        
    try:
        storage=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(6) > div > span').text
        Storage_rating.append(storage)
    except NoSuchElementException:
        Storage_rating.append("-")
        
    try:
        overall=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(7) > div > span').text
        Overall_rating.append(overall)
    except NoSuchElementException:
        Overall_rating.append("-")
        
        
    try:
        availibility=driver.find_element_by_css_selector('#price > div > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text
        Availibility.append(availibility)
    except NoSuchElementException:
        Availibility.append("-")
        
    try:
        digit=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.score-Bar > ul > li:nth-child(1) > p').text
        Digit_rating.append(digit)
    except NoSuchElementException:
        Digit_rating.append("-")
        
    try:
        user=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.score-Bar > ul > li:nth-child(2) > p').text
        User_rating.append(user)
    except NoSuchElementException:
        User_rating.append("-")
        
    try:
        pros=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.wrap-rat > div > div.col-md-4 > div > ul:nth-child(1)').text.replace("\n", " ")
        Pros.append(pros)
    except NoSuchElementException:
        Pros.append("-")
        
    try:
        cons=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.wrap-rat > div > div.col-md-4 > div > ul:nth-child(2)').text.replace("\n", " ")
        Cons.append(cons)
    except NoSuchElementException:
        Cons.append("-")
        
    time.sleep(10)

100%|██████████| 10/10 [01:53<00:00, 11.33s/it]


In [382]:
Market_status

['LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED']

In [383]:
Release_date

['10-Aug-2019',
 '01-Dec-2019',
 '21-Jul-2020',
 '12-Aug-2020',
 '15-Jul-2019',
 '30-Aug-2019',
 '31-May-2019',
 '08-Jan-2017',
 '20-Nov-2020',
 '23-Oct-2018']

In [384]:
Official_website

['MSI', 'Dell', 'HP', 'Asus', 'Lenovo', 'Asus', 'Asus', 'MSI', 'Asus', 'Dell']

In [385]:
Display_rating

['85', '85', '85', '-', '85', '-', '85', '85', '-', '85']

In [386]:
CPU_rating

['82', '82', '65', '-', '64', '-', '59', '52', '-', '59']

In [387]:
GPU_rating

['51', '51', '38', '-', '41', '-', '51', '51', '-', '28']

In [388]:
RAM_rating

['96', '142', '92', '-', '116', '-', '75', '142', '-', '92']

In [389]:
Storage_rating

['45', '45', '30', '-', '65', '-', '59', '75', '-', '56']

In [390]:
Overall_rating

['72', '81', '72', '-', '57', '-', '66', '81', '-', '74']

In [391]:
Availibility

['-',
 'Available',
 'Available',
 'Available',
 'Out of Stock',
 'Out of Stock',
 'Available',
 '-',
 'Available',
 'Available']

In [392]:
Digit_rating

['-', '-', '77', '71', '74', '82', '3.2', '-', '-', '67']

In [393]:
User_rating

['-', '-', '3.4', '4.6', '4.3', '4.8', '-', '-', '-', '4.4']

In [394]:
Pros

['-',
 '-',
 'Pros + Impressive thermals. + Performs well for gamers and content creators. + The display is crisp and colour accurate',
 'Pros + Impressive performance for the form-factor. + Anime Matrix display is a very well implemented gimmick. + Truly one-of-a-kind machine at the moment.',
 'Pros + Sturdy build, clean look + Top performance (incl. gaming) + Decent audio quality',
 'Pros + Noteworthy performance + Comfortable keyboard + Surprisingly good audio',
 '-',
 '-',
 '-',
 'Pros + Seriously fast fingerprint scanner + Speakers are quite loud + Stays cool when clock rates rise']

In [395]:
Cons

['-',
 '-',
 'Cons - The lid wobbles and flexes. - Nvidia GTX 1650Ti feels a little out of place in this config.',
 'Cons - Heat makes the keyboard unusable. - Uses a mid-range SSD. - No Pg Up or Pg Dn keys',
 'Cons - Abysmally short battery life - Oddly placed keys - Rear ports are inconvenient',
 'Cons - Noisy fans can distract - Display could use more colour - No inbuilt webcam',
 '-',
 '-',
 '-',
 'Cons - Bulky at 2.53 kilogrammes - Keys lack sufficient travel - Colours appear washed out']

In [397]:
#printing lengths
print(len(Laptop_name),len(Seller),len(PRICE),len(Operating_system),len(Display),len(Processor),len(Memory),len(Weight),len(Graphics),len(Description),len(Market_status),len(Release_date),len(Official_website),len(Display_rating),len(CPU_rating),len(GPU_rating),len(RAM_rating),len(Storage_rating),len(Overall_rating),len(Availibility),len(Digit_rating),len(User_rating),len(Pros),len(Cons),len(Laptop_URL))

10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10


In [398]:
#creating a dataframe

Top10_Laptops=pd.DataFrame({})
Top10_Laptops['Laptop_name']=Laptop_name
Top10_Laptops['Seller']=Seller
Top10_Laptops['Price']=PRICE
Top10_Laptops['Operating_system']=Operating_system
Top10_Laptops['Display']=Display
Top10_Laptops['Processor']=Processor
Top10_Laptops['Memory']=Memory
Top10_Laptops['Weight']=Weight
Top10_Laptops['Graphics']=Graphics
Top10_Laptops['Description']=Description
Top10_Laptops['Market_status']=Market_status
Top10_Laptops['Release_date']=Release_date
Top10_Laptops['Official_website']=Official_website
Top10_Laptops['Display_rating']=Display_rating
Top10_Laptops['CPU_rating']=CPU_rating
Top10_Laptops['GPU_rating']=GPU_rating
Top10_Laptops['RAM_rating']=RAM_rating
Top10_Laptops['Storage_rating']=Storage_rating
Top10_Laptops['Overall_rating']=Overall_rating
Top10_Laptops['Availibility']=Availibility
Top10_Laptops['Digit_rating']=Digit_rating
Top10_Laptops['User_rating']=User_rating
Top10_Laptops['Pros']=Pros
Top10_Laptops['Cons']=Cons
Top10_Laptops['Laptop_URL']=Laptop_URL

In [400]:
#to display full dataframe
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [401]:
Top10_Laptops

,Laptop_name,Seller,Price,Operating_system,Display,Processor,Memory,Weight,Graphics,Description,Market_status,Release_date,Official_website,Display_rating,CPU_rating,GPU_rating,RAM_rating,Storage_rating,Overall_rating,Availibility,Digit_rating,User_rating,Pros,Cons,Laptop_URL
0,MSI GT76 TITAN DT 9SG,N/A,₹379990,WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,1 TB HDD/64 GBGB DDR4,4.2 kg,NVIDIA GeForce RTX 2080,Sporting a desktop graphics Intel Core i9-9900...,LAUNCHED,10-Aug-2019,MSI,85,82,51,96,45,72,-,-,-,-,-,https://www.digit.in/laptops/msi-gt76-titan-dt...
1,ALIENWARE 17 AREA-51M,amazon,₹489909,WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,1 TB PCIE SSD/32GB DDR4,3.87 Kg,NVIDIA GeForce RTX 2080,The Alienware Area-51m was certainly the first...,LAUNCHED,01-Dec-2019,Dell,85,82,51,142,45,81,Available,-,-,-,-,https://www.digit.in/laptops/alienware-area-51...
2,HP OMEN 15 2020,Tatacliq,₹117790,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",INTEL I7-10750H 10TH GEN | 1.6GHZ,512 GB SSD/16 GBGB DDR4,5.40,Nvidia GeForce GTX 1650Ti,The new HP Omen 15 comes in both 10th generati...,LAUNCHED,21-Jul-2020,HP,85,65,38,92,30,72,Available,77,3.4,Pros + Impressive thermals. + Performs well fo...,Cons - The lid wobbles and flexes. - Nvidia GT...,https://www.digit.in/laptops/hp-omen-15-2020-p...
3,ASUS ZEPHYRUS G14,amazon,₹164990,WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,1.65,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...,LAUNCHED,12-Aug-2020,Asus,-,-,-,-,-,-,Available,71,4.6,Pros + Impressive performance for the form-fac...,Cons - Heat makes the keyboard unusable. - Use...,https://www.digit.in/laptops/asus-rog-zephyrus...
4,LENOVO LEGION Y540,flipkart,₹79990,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ,1 TB SSD/8GB DDR4,2.3,NVIDIA® GeForce RTX™ 2060,The Lenovo Legion Y540 is powered by an Intel ...,LAUNCHED,15-Jul-2019,Lenovo,85,64,41,116,65,57,Out of Stock,74,4.3,"Pros + Sturdy build, clean look + Top performa...",Cons - Abysmally short battery life - Oddly pl...,https://www.digit.in/laptops/lenovo-legion-y54...
5,ASUS ROG ZEPHYRUS G GA502,flipkart,₹79990,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ,512 GB SSD/16GB DDR4,2.2,NVIDIA Geforce GTX 1660 Ti,The Ryzen 7 3750H powered Asus ROG Zephyrus G ...,LAUNCHED,30-Aug-2019,Asus,-,-,-,-,-,-,Out of Stock,82,4.8,Pros + Noteworthy performance + Comfortable ke...,Cons - Noisy fans can distract - Display could...,https://www.digit.in/laptops/asus-rog-zephyrus...
6,ASUS ROG ZEPHYRUS S GX531,amazon,₹146000,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",9TH GEN INTEL CORE I7-8750H | 2.2 GHZ,512GB SSD/16 GBGB DDR4,2.1,NVIDIA® GeForce RTX™ 2070 (Max-Q),The Asus Zephyrus S (GX531) manages to get a f...,LAUNCHED,31-May-2019,Asus,85,59,51,75,59,66,Available,3.2,-,-,-,https://www.digit.in/laptops/asus-rog-zephyrus...
7,MSI GT83VR 7RE TITAN SLI,N/A,₹349990,WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ,1.5 TB SATA/64GB DDR4,5.5,Dual GTX1070,MSI does not have a dual GTX 1080 gaming lapto...,LAUNCHED,08-Jan-2017,MSI,85,52,51,142,75,81,-,-,-,-,-,https://www.digit.in/laptops/msi-gt83vr-7re-ti...
8,ASUS ROG ZEPHYRUS DUO 15,amazon,₹268990,WINDOWS 10,"15.6"" (3840 X 1100)",INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...,LAUNCHED,20-Nov-2020,Asus,-,-,-,-,-,-,Available,-,-,-,-,https://www.digit.in/laptops/asus-rog-zephyrus...
9,DELL G3,amazon,₹61990,WINDOWS 10 HOME PLUS,15.6 MP | NA,8TH GENERATION INTEL® CORE™ I7-8750H | NA,256 + 1 TB SSD + SATA/16GB DDR4,2.53,NVIDIA® GeForce® GTX,The Dell G3 is a capable entry-level gaming la...,LAUNCHED,23-Oct-2018,Dell,85,59,28,92,56,74,Available,67,4.4,Pros + Seriously fast fingerprint scanner + Sp...,Cons - Bulky at 2.53 kilogrammes - Keys

In [402]:
#converting into csv format
Top10_Laptops.to_csv("Top10_Laptops.csv")

In [403]:
#closing the driver
driver.close()